In [1]:
from MySQLdb import connect
from dask.distributed import Client
import dask.dataframe as dd
import witxcraft as wc
import pandas as pd
import plotly.express as px
import statsmodels

client = Client(n_workers=8)

SINGLESTORE_HOST = "172.17.0.1"
SINGLESTORE_PORT = 3306

SINGLESTORE_URI = "mysql+mysqldb://root:root@{}:{}/demo_big".format(SINGLESTORE_HOST, SINGLESTORE_PORT)

distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-0_2hjxh8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-sdps97tn', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-40bl6vz0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-evlnzqj0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-lf4_8fke', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-ij5q5mf0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/exa

In [2]:
df = dd.read_sql_table('comments_small', SINGLESTORE_URI, npartitions=8, index_col='id')
df.head()

/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  KEY `__UNORDERED` () USING CLUSTERED COLUMNSTORE'
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)
/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  , SHARD KEY () '
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)


,text,creation_date,score
id,,,
85673569,buffer=(char *)malloc(fileLen+1);\n if (!buffe...,2018-03-17 12:39:42,0
85559717,é voila... works. Used `patch apply /Users/dan...,2018-03-14 13:11:29,0
85472668,"sorry for miscommunication ,I got variable arg...",2018-03-12 12:05:57,0
85473382,Not enough information. Where is the rest of y...,2018-03-12 12:23:45,0
85599569,"try `<textarea>` instead, or some WYSIWYG text...",2018-03-15 12:02:34,0


In [3]:
len(df)

1000

In [4]:
ml = wc.fromwasmmod('example_wasm.wasm', witx='iface.witx')
ml.sentiment('covid is bad')

compound   -0.542326
positive    0.000000
negative    0.636364
neutral     0.363636
dtype: float64

In [5]:
out = df['text'].apply(ml.sentiment, meta=ml.sentiment.ret_meta)
out.head()

,compound,positive,negative,neutral
id,,,,
85673569,-0.507974,0.000000,0.190002,0.809998
85559717,-0.367150,0.055316,0.140091,0.804593
85472668,-0.077228,0.000000,0.047619,0.952381
85473382,-0.240590,0.000000,0.000000,1.000000
85599569,0.000000,0.000000,0.000000,1.000000


In [21]:
from pymysql import connect, cursors
s2 = connect(host=SINGLESTORE_HOST, port=SINGLESTORE_PORT, user='root', passwd='root', db='demo_big', local_infile=True, cursorclass=cursors.DictCursor)
cursor = s2.cursor()

In [22]:
cursor.execute("select count(*) as cnt from comments")
cursor.fetchone()["cnt"]

82037742

In [23]:
cursor.execute("""
    create or replace function sentiment_vec(input text not null)
    returns table(
        compound double,
        positive double,
        negative double,
        neutral double
    ) as wasm infile '/workspaces/wasi-data-python/examples/example_wasm.wasm';
""")

1

In [24]:
cursor.execute("select * from sentiment_vec('hello wasm day! I love wasm!')")
cursor.fetchall()

[{'compound': 0.6988421622775413,
  'positive': 0.5446265938069217,
  'negative': 0.0,
  'neutral': 0.45537340619307837}]

In [25]:

df = pd.read_sql("""
    select
        (score - (score %% 10)) as score_bucket,
        count(*) as num_comments,
        abs(min(s.compound)) as "negative",
        max(s.compound) as "positive"
    from
        (select * from comments where score >= 10) as c,
        sentiment_vec(c.text) as s
    group by 1
    having positive > 0 and negative > 0 and num_comments > 20
    order by 1 asc
""", con=SINGLESTORE_URI)

df.head()

,score_bucket,num_comments,negative,positive
0,10,115285,0.999408,0.988480
1,20,27409,0.988882,0.989240
2,30,12064,0.982151,0.987701
3,40,6701,0.984391,0.987944
4,50,4130,0.957839,0.977145


In [26]:
colorscheme = ["#00E676" ,"#AA00FF"]

fig = px.scatter(df,
    x="score_bucket",
    y=["positive", "negative"],
    trendline="ols",                        # ols = Ordinary Least Squares
    color_discrete_sequence=colorscheme,
    template="plotly_dark",
    labels={
        "score_bucket": "Comment Score",
        "value": "sentiment",
    }
)

fig.update_layout(
    paper_bgcolor='#1B1A21',
    plot_bgcolor='#1B1A21',
)

fig.show()